In [1]:
from pprint import pprint
import pandas as pd
from math import ceil
df = pd.read_json('./2021_1_data.json')

In [2]:
def get_tut_details(index):
    result = []
    for i in index:
        details = i['details']
        for d in details:
            day = d['day']
            room = d['location']
            time = d['time']['start']
            duration = ceil(d['time']['duration'])

            if d['type'] == 'TUT' and room != 'Online' and room != 'ONLINE' and room != '':
                result.append({'room': room, 'day': day, 'start_time': time, 'duration': duration})
    return result

In [3]:
total_result = []
for course_code in df:
    total_result.extend(get_tut_details(df[course_code]['index']))

In [4]:
group_by_day = {'MON': {}, 'TUE': {}, 'WED': {}, 'THU': {}, 'FRI': {}}
for data in total_result:
    room = data['room']
    day = data['day']
    free_time_slots = [True for _ in range(14)]

    group_by_day[day][room] = free_time_slots

In [7]:
import json
with open('room_datastructure.json', 'w') as outfile:
    json.dump(group_by_day, outfile, indent=2)

In [6]:
start_time_to_idx = {'0830': 0, '0930': 1, '1030': 2, '1130': 3, '1230': 4, '1330': 5, '1430': 6, '1530': 7, '1630': 8, '1730': 9, '1830': 10, '1930': 11, '2030': 12, '2130': 13, '2230': 14}

for data in total_result:
    room, day, start_time, duration = data.values()
    from_idx = start_time_to_idx[start_time]

    for index, value in enumerate(group_by_day[day][room]):
        for i in range(from_idx, from_idx+duration):
            group_by_day[day][room][i] = False

In [8]:
import json
with open('room_occupancy.json', 'w') as outfile:
    json.dump(group_by_day, outfile, indent=2)